Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Goal

This notebook creates a real-time scoring service for the content-personalization model created in the prior [notebook](../02_model/mmlspark_lightgbm_criteo.ipynb). It is assumed that this notebook is run in an Azure Databricks environment that has had `mmlspark` installed and has been prepared for operationalization. See [Setup instructions](https://github.com/Microsoft/Recommenders/blob/master/SETUP.md) for details.

**NOTE**: Please Register Azure Container Instance (ACI) using Azure Portal: https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-manager-supported-services#portal in your subscription before using the SDK to deploy your ML model to ACI.

### Setup libraries and variables

The next few cells initialize the environment and varibles: we import relevant libraries and set variables.

In [ ]:
import os
import uuid
import json

from azureml.core import Workspace
from azureml.core import VERSION as amlversion

from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.webservice import Webservice, AciWebservice
from azureml.core.image import ContainerImage

# Check core SDK version number
print("SDK version:", amlversion)

In [ ]:
# These variables are used to construct names of assets:
short_uuid = str(uuid.uuid4())[:4]
prefix = "reco" + short_uuid
data = "criteo"
algo = "lgbm"

In [ ]:
# Azure subscription
subscription_id = ''

# Resource group and workspace
resource_group = prefix + "_" + data
workspace_name = prefix + "_"+data+"_aml"
workspace_region = "westus2"
print("Resource group:", resource_group)

# AzureML
#NOTE: The name of a asset must be only letters or numerals, not contain spaces, and under 30 characters
model_name = data+"-"+algo+".model" 
service_name = data + "-" + algo

# add a name for the container
container_image_name = '-'.join([data, algo])


## locations for serializing so it persists. This is a local API URL
ws_config_path = '/dbfs/FileStore'
## location of model on **dbfs**:
model_path = os.path.join('dbfs:/FileStore/dac',model_name)
## path to the notebook for modeling. Assumes the entire repository has been imported:
modeling_notebook = '../02_model/mmlspark_lightgbm_criteo'

## names of other files that are used below
my_conda_file = "deploy_conda.yml"
driver_file = "score_sparkml.py"


## Prepare Assets for the Scoring Service

Before walking through the steps taken to create a model, it is useful to set some context. In our example, a "scoring service" is a function that is executed by a docker container. It takes in some number of records and produces a set of scores for each record (usually predictions of some type) based on a previously estimated model. In our case, we will take the model we estimated earlier that predicts the probability of a click based on some set of numeric and categorical features. In order to create a scoring service, we will do several steps.

We will:

1. Create an Azure Machine Learning Workspace to simplify all the subsequent steps.
2. Make sure we have access to the previously estimated model. If we are working on a spark system, that means we will make sure the model is on the local filesystem (**not** DBFS) and registered with the Azure Machine Learning Service.
3. Define a 'driver' script that defines what the system needs to do in order to generate our predictions. This script needs to have an `init` method that does one-time initialization and a `run` method that is executed each time the service is called.
4. Define all the pre-requisites that that script requries.
5. Use the model, the driver script, and the pre-requisites to create a docker image.
6. We will run the docker image on a platform (in our case Azure Container Instance or ACI).
7. We will test our service.

## Create a Workspace

In [ ]:
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      exist_ok=True)

# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config(ws_config_path)

## 2. Prepare the Serialized Model

First, we will prepare the serialized model. We will make sure the model exists, and if it doesn't, then we will run the notebook to generate the file.

In [ ]:
## if it doesn't exist, run the relevant notebook:
if not os.path.exists(model_path.replace('dbfs:','/dbfs')):
  print('Model pipeline does not exist. Creating by running {}'.format(modeling_notebook))
  dbutils.notebook.run(modeling_notebook, timeout_seconds=600)
else:
  print('Operationalizing model found at: {}'.format(model_path))

### Copy from dbfs to local

While you can access files on DBFS with local file APIs, it is better practice to explicitly copy saved models to and from dbfs, because the local file APIs can only access files smaller than 2 GB (see details [here](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html#access-dbfs-using-local-file-apis)).  

Model deployment will always get the model from the current working directory.

In [ ]:
model_local = "file:" + os.getcwd() + "/" + model_name
dbutils.fs.cp(model_path, model_local, True)

### Register the Model

Next, we need to register the model in the Azure Machine Learning Workspace.

In [ ]:
#Register the model
mymodel = Model.register(model_path = model_name, # this points to a local file
                       model_name = model_name, # this is the name the model is registered as
                       description = "LightGBM Criteo Model",
                       workspace = ws)

print(mymodel.name, mymodel.description, mymodel.version)

## 3. Create the Driver Script

Next we, need to create the driver script that will be executed when the service is called. The functions that need to be defined for scoring are `init()` and `run()`. The `init()` function is run when the service is created, and the `run()` function is run each time the service is called.

In our example, we use the `init()` function to load all the libraries, initialize the spark session, and load the model and pipeline. We use the `run()` method to parse the input json file, generate predictions (in this case the probability of a click), and format for output.

In [ ]:
score_sparkml = """

import json
 
def init():
    # One-time initialization of PySpark and predictive model
    import pyspark
    from pyspark.ml import PipelineModel
    from mmlspark import LightGBMClassifier
    from azureml.core.model import Model
    from pyspark.ml import PipelineModel
    from pyspark.sql.types import StructType, StructField, IntegerType, StringType

    global trainedModel
    global spark
    global schema

    spark = pyspark.sql.SparkSession.builder.appName("LightGBM Criteo Predictions").getOrCreate()
    model_name = "{model_name}" 
    model_path = Model.get_model_path(model_name)
    trainedModel = PipelineModel.load(model_path)
    
def run(input_json):
    if isinstance(trainedModel, Exception):
        return json.dumps({{"trainedModel":str(trainedModel)}})
      
    try:
        sc = spark.sparkContext
        input_list = json.loads(input_json)
        input_rdd = sc.parallelize(input_list)
        input_df = spark.read.json(input_rdd)
        
        # Compute prediction
        predictions = trainedModel.transform(input_df).collect()
        #Get probability of a click for each row and conver to a str
        click_prob = [str(x.probability[1]) for x in predictions]

        # you can return any data type as long as it is JSON-serializable
        result = ",".join(click_prob)
        return [result]
    except Exception as e:
        result = str(e)
        return result
""".format(model_name=model_name)
 
exec(score_sparkml)
 
with open(driver_file, "w") as file:
    file.write(score_sparkml)

## 4. Define Dependencies

Next, we define the dependencies that are required by driver script.

In [ ]:
## azureml-sdk is required to load the registered model
myconda = CondaDependencies.create(pip_packages=['azureml-sdk'])
with open(my_conda_file,"w") as f:
    f.write(myconda.serialize_to_string())

## 5. Create the Image

We use the `ContainerImage` class to first configure, then to create the docker image used. 

In [ ]:
myimage_config = ContainerImage.image_configuration(execution_script = driver_file, 
                                                    runtime = "spark-py",
                                                    conda_file=my_conda_file,
                                                    tags={"runtime":"pyspark", "algorithm":"lightgbm"})

image = ContainerImage.create(name = service_name,
                              models = [mymodel],
                              image_config = myimage_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

## 6. Create the Service

Once we have created an image, we configure and run it on ACI.

**NOTE** You *can* create a service directly from the registered model and image_configuration with the `Webservice.deploy_from_model()` function. We create the image here explicitly and use `deploy_from_image()` for two reasons:

1. It provides more transparency in terms of the actual steps that are taking place
2. It has potential for faster iteration and for more portability. Once we have an image, we can create a new deployment with the exact same code.

In [ ]:
#configure ACI
myaci_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 2, 
    tags = {'name':'Azure ML ACI for LightGBM', 'algorithm':'LightGBM'}, 
    description = 'Light GBM ACI.')

# Webservice creation
myservice = Webservice.deploy_from_image(
  workspace=ws, 
  name=service_name,
  image=image,
  deployment_config = myaci_config
    )

myservice.wait_for_deployment(show_output=True)

### View the URI

In [ ]:
#for using the Web HTTP API 
print(myservice.scoring_uri)

## 7. Test the Service

Next, we can use data from the `test` data to test the service.

The service expects JSON as its payload, so we take the test data, fill missing values, convert to JSON, then submit to the service endpoint.

We have to fill in missing values here to create the data, because the webservice expects that the data coming into the webservice is well-formed. 

In [ ]:
n_samples_to_test = 10

## load the table created in the other notebook:
test=spark.table('test')
test_for_service_df = test.drop('features').fillna('M').fillna(0).limit(n_samples_to_test)
display(test_for_service_df)
test_json = json.dumps(test_for_service_df.toJSON().collect())

### Run the Service and Parse the Output

In [ ]:
## The prediction is the predicted probability of a click for that particular record
service_out = myservice.run(input_data=test_json)
print(service_out)
values=json.loads('['+service_out[0]+']')

### Delete the Service

When you are done, you can delete the service to minimize costs. You can always redeploy from the image using the same command above.

In [ ]:
## Uncomment the following line to delete the web service
# myservice.delete()

## Additional Resources

- See the notebook for model estimation [here](https://github.com/Microsoft/Recommenders/blob/gramhagen/lgbm_scenario/notebooks/02_model/mmlspark_lightgbm_criteo.ipynb).
- This notebook is adapted from the notebooks [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/azure-databricks/amlsdk/).
- See an example of leveraging the image on AKS [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/azure-databricks/amlsdk/deploy-to-aks-existingimage-05.ipynb).
